# Text Wrangling and Regex

Working with text: applying string methods and regular expressions

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import pandas as pd

## Demo 1: Canonicalizing County Names

Load both **county_and_state.csv** and **county_and_population.csv**

In [2]:
# display both frames
#code Here
df1=pd.read_csv("county_and_state.csv")
df1

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


In [3]:
df2=pd.read_csv("county_and_population.csv")
df2

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


Both of these DataFrames share a "County" column. Unfortunately, formatting differences mean that we can't directly merge the two DataFrames using the "County"s.

In [4]:
#merge both frames
pd.merge(df1,df2,on="County")

,County,State,Population


To address this, we can **canonicalize** the "County" string data to apply a common formatting.

In [5]:
def canonicalize_county(county_series):
    return (county_series.str.lower().str.replace(" ","").str.replace("&","and").str.replace(".","").str.replace("county","").str.replace("parish",""))


Apply canonicalize_county on 'County' columns in both frames.

In [6]:
# Code Here
df1['County'] = canonicalize_county(df1['County'])
df2['County'] = canonicalize_county(df2['County'])


Now, the merge works as expected!

In [7]:
# code Here
pd.merge(df1,df2,on="County")

,County,State,Population
0,dewitt,IL,16798
1,lacquiparle,MN,8067
2,lewisandclark,MT,55716
3,stjohnthebaptist,LS,43044


## Demo 2: Extracting Log Data

Load log.txt

In [8]:
#code Here
        # Process each line as needed

with open("log.txt") as file:
    data=file.readlines()

data

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [9]:
# code here
data[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [10]:
data

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

In [11]:
a='''169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n' '''

[('26', 'Jan', '2014', '10', '47', '58', '0800'),
 ('2', 'Feb', '2005', '17', '23', '6', '0800'),
 ('3', 'Feb', '2006', '10', '18', '37', '0800')]

In [14]:
pattren=r"\[[0-9]+\/[A-Za-z]{3}\/[0-9]{4}:[0-9]{2}:[0-9]+:[0-9]+\s-[0-9]{4}\]"
re.findall(pattren,a)

['[26/Jan/2014:10:47:58 -0800]',
 '[2/Feb/2005:17:23:6 -0800]',
 '[3/Feb/2006:10:18:37 -0800]']

Apply string functions of python to extract date from first entry in log file.

In [15]:
                                        # find the text enclosed in square brackets
                                        # split up the date/month/year
                                        # split up the hour:minute:second
                                        # split the timezone after the blank space
import re
re.findall(r"\[(\d+)\/(\w+)\/(\d+):(\d+):(\d+):(\d+)\s+-(\d+)\]",a)

[('26', 'Jan', '2014', '10', '47', '58', '0800'),
 ('2', 'Feb', '2005', '17', '23', '6', '0800'),
 ('3', 'Feb', '2006', '10', '18', '37', '0800')]

This worked, but felt fairly "hacky" – the code above isn't particularly elegant. A much more sophisticated but common approach is to extract the information we need using a *regular expression*.


# Regular Expressions


## String Extraction with Regex

Python `re.findall` returns a list of all extracted matches: extract numbers from given string.

In [48]:
import re

text = "My social security number is 123-45-6789 bro, or actually maybe it’s 321-45-6789.";
# code Here

pi=r"([0-9]{3})-([0-9]{2})-([0-9]{4})"
t1=re.findall(pi,text)
t1

[('123', '45', '6789'), ('321', '45', '6789')]

<br/>

Now, let's see vectorized extraction in `pandas`:

 `.str.findall` returns a `Series` of lists of all matches in each record.

In [40]:
data = ['987-65-4321','forty','123-45-6789 bro or 321-45-6789','999-99-9999']
# code here
df=pd.DataFrame(data,columns=["SSn"])
df

,SSn
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


Find all entries containing numbers in dataframe.

In [41]:
# -> Series of lists
# code Here
df["SSn"]

0                       987-65-4321
1                             forty
2    123-45-6789 bro or 321-45-6789
3                       999-99-9999
Name: SSn, dtype: object

## Extraction Using Regex Capture Groups

The Python function `re.findall`, in combination with parentheses returns specific substrings (i.e., **capture groups**) within each matched string, or **match**.

In [42]:

# code here
text = """I will meet you at 08:30:00 pm tomorrow"""
i= r"(\d+):(\d+):(\d+)"
re.findall(i,text)

[('08', '30', '00')]

In [ ]:
# the three capture groups in the first matched string
hour, minute, second = matches[0]

<br/>

In `pandas`, we can use `.str.extract` to extract each capture group of **only the first match** of each record into separate columns.

In [43]:
# back to SSNs
df

,SSn
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


In [50]:
# Will extract the first match of all groups
# code here
df["SSn"].str.extract(pi)

,0,1,2
0,987,65,4321
1,NaN,NaN,NaN
2,123,45,6789
3,999,99,9999


Alternatively, `.str.extractall` extracts **all matches** of each record into separate columns. Rows are then MultiIndexed by original record index and match index.

In [52]:
# -> DataFrame, one row per match
# code Here
df["SSn"].str.extractall(pi)

0   1     2
  match               
0 0      987  65  4321
2 0      123  45  6789
  1      321  45  6789
3 0      999  99  9999

## Canonicalization with Regex

In regular Python, canonicalize with `re.sub` (standing for "substitute"):

In [73]:
text = '<div><td valign="top">Moo</td></div>'

#code here

o=re.sub(r"<[^>]+>","",text)
o

'Moo'

<br/>

In `pandas`, canonicalize with `Series.str.replace`.

In [70]:
df_html = ['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>']

ty=pd.DataFrame(df_html,columns=["HTML"])
ty

,HTML
0,"<div><td valign=""top"">Moo</td></div>"
1,"<a href=""http://ds100.org"">Link</a>"
2,<b>Bold text</b>


In [72]:
# example dataframe of strings, convert in dataframe
df_html = ['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>']


0    <div><td valign="top">Moo</td></div>
1     <a href="http://ds100.org">Link</a>
2                        <b>Bold text</b>
Name: HTML, dtype: object

In [75]:
pp=r"<[^>]+>"
ty["HTML"].str.replace(pp," ")

0    <div><td valign="top">Moo</td></div>
1     <a href="http://ds100.org">Link</a>
2                        <b>Bold text</b>
Name: HTML, dtype: object

In [ ]:
# Series -> Series
#Extract only words



# Revisiting Text Log Processing using Regex

### Python `re` version

In [1]:
with open("log.txt") as file:
    log_lines=file.readlines()
line = log_lines[0]
display(line)
 

### `pandas` version

In [ ]:
# code here


,Log
0,169.237.46.168 - - [26/Jan/2014:10:47:58 -0800...
1,"193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] ""..."
2,"169.237.46.240 - """" [3/Feb/2006:10:18:37 -0800..."


Option 1: `Series.str.findall`

In [ ]:
# code here
re.findall(r"\[(\d+)\/(\w+)\/(\d+):(\d+):(\d+):(\d+)\s+-(\d+)\]",a)

0    [(26, Jan, 2014, 10, 47, 58, -0800)]
1      [(2, Feb, 2005, 17, 23, 6, -0800)]
2     [(3, Feb, 2006, 10, 18, 37, -0800)]
Name: Log, dtype: object

<br/>

Option 2: `Series.str.extractall`

In [ ]:
# code here
df["log"].extractall(r1,data)

,,0,1,2,3,4,5,6
,match,,,,,,,
0,0,26,Jan,2014,10,47,58,-0800
1,0,2,Feb,2005,17,23,6,-0800
2,0,3,Feb,2006,10,18,37,-0800


Wrangling either of these two DataFrames into a nice format (like below) is left as an exercise for you!


||Day|Month|Year|Hour|Minute|Second|Time Zone|
|---|---|---|---|---|---|---|---|
|0|26|Jan|2014|10|47|58|-0800|
|1|2|Feb|2005|17|23|6|-0800|
|2|3|Feb|2006|10|18|37|-0800|


In [ ]:
# your code here
...